# Análise Exploratória de Dados - Risco de Crédito

Este notebook realiza uma análise exploratória dos dados de risco de crédito para entender melhor as distribuições, correlações e padrões nos dados antes de iniciar o processo de modelagem.

## Importação das Bibliotecas

In [ ]:
# Bibliotecas básicas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Configurações de visualização
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12
sns.set_palette('viridis')

# Ignorar avisos
import warnings
warnings.filterwarnings('ignore')

# Carregar funções próprias do projeto
import sys
sys.path.append('..')
from src.data_processing import load_data_from_csv, handle_missing_values

## Carregamento dos Dados

In [ ]:
# Carregar dados brutos
data_path = '../data/raw/credit_data.csv'
df = load_data_from_csv(data_path)

# Exibir as primeiras linhas para verificação
df.head()

## Análise Básica dos Dados

In [ ]:
# Dimensões do dataset
print(f"Shape do Dataset: {df.shape}")

# Informações das colunas
print("\nInformações das Colunas:")
df.info()

In [ ]:
# Estatísticas descritivas
df.describe().T

## Análise de Valores Ausentes

In [ ]:
# Verificar valores ausentes
missing_values = df.isnull().sum()
missing_percentage = (missing_values / len(df)) * 100

missing_df = pd.DataFrame({
    'Valores Ausentes': missing_values,
    'Porcentagem (%)': missing_percentage
}).sort_values('Valores Ausentes', ascending=False)

# Exibir apenas colunas com valores ausentes
missing_df[missing_df['Valores Ausentes'] > 0]

In [ ]:
# Visualizar valores ausentes
plt.figure(figsize=(12, 6))
sns.heatmap(df.isnull(), cmap='viridis', cbar=False, yticklabels=False)
plt.title('Mapa de Valores Ausentes')
plt.tight_layout()
plt.show()

## Distribuição da Variável Alvo

In [ ]:
# Definir variável alvo (ajuste conforme seu dataset)
target_col = 'risco_credito'

# Contagem da variável alvo
plt.figure(figsize=(10, 6))
target_counts = df[target_col].value_counts()
ax = sns.barplot(x=target_counts.index, y=target_counts.values)

# Adicionar rótulos com porcentagens
total = len(df)
for p in ax.patches:
    percentage = f'{100 * p.get_height() / total:.1f}%'
    ax.annotate(percentage, (p.get_x() + p.get_width() / 2., p.get_height()),
                ha='center', va='center', xytext=(0, 10), textcoords='offset points')

plt.title('Distribuição da Variável Alvo: Risco de Crédito')
plt.xlabel('Categoria de Risco')
plt.ylabel('Contagem')
plt.xticks(rotation=0)
plt.show()

## Análise de Variáveis Numéricas

In [ ]:
# Identificar colunas numéricas
numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns.tolist()
numeric_cols = [col for col in numeric_cols if col != target_col]  # Excluir target

# Selecionar até 6 colunas numéricas para visualização
display_cols = numeric_cols[:6] if len(numeric_cols) > 6 else numeric_cols

# Histogramas das principais variáveis numéricas
fig, axes = plt.subplots(2, 3, figsize=(18, 10))
axes = axes.flatten()

for i, col in enumerate(display_cols):
    sns.histplot(data=df, x=col, kde=True, ax=axes[i])
    axes[i].set_title(f'Distribuição de {col}')
    axes[i].set_ylabel('Frequência')
    
plt.tight_layout()
plt.show()

In [ ]:
# Box plots para detectar outliers
fig, axes = plt.subplots(2, 3, figsize=(18, 10))
axes = axes.flatten()

for i, col in enumerate(display_cols):
    sns.boxplot(x=df[col], ax=axes[i])
    axes[i].set_title(f'Box Plot de {col}')
    
plt.tight_layout()
plt.show()

## Análise de Variáveis Categóricas

In [ ]:
# Identificar colunas categóricas
categorical_cols = df.select_dtypes(include=['object', 'category']).columns.tolist()
categorical_cols = [col for col in categorical_cols if col != target_col]  # Excluir target se for categórico

# Contagem de valores para cada categoria
for col in categorical_cols[:4]:  # Limitando a 4 colunas para melhor visualização
    plt.figure(figsize=(12, 6))
    value_counts = df[col].value_counts().sort_values(ascending=False)
    
    # Limitar a 10 categorias para melhor visualização
    if len(value_counts) > 10:
        value_counts = value_counts[:10]
        title_suffix = " (Top 10)"
    else:
        title_suffix = ""
    
    sns.barplot(x=value_counts.index, y=value_counts.values)
    plt.title(f'Distribuição da Variável: {col}{title_suffix}')
    plt.xlabel(col)
    plt.ylabel('Contagem')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()

## Análise Bivariada com a Variável Alvo

In [ ]:
# Relação entre variáveis numéricas e a variável alvo
for col in display_cols:
    plt.figure(figsize=(12, 6))
    sns.boxplot(x=target_col, y=col, data=df)
    plt.title(f'Relação entre {col} e {target_col}')
    plt.tight_layout()
    plt.show()

In [ ]:
# Relação entre variáveis categóricas e a variável alvo
for col in categorical_cols[:3]:  # Limitando a 3 colunas
    plt.figure(figsize=(14, 7))
    
    # Limitar a 6 categorias mais frequentes
    top_categories = df[col].value_counts().nlargest(6).index
    plot_df = df[df[col].isin(top_categories)]
    
    cross_tab = pd.crosstab(plot_df[col], plot_df[target_col], normalize='index')
    cross_tab.plot(kind='bar', stacked=True)
    
    plt.title(f'Relação entre {col} e {target_col}')
    plt.xlabel(col)
    plt.ylabel('Proporção')
    plt.legend(title=target_col)
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()

## Correlação entre Variáveis Numéricas

In [ ]:
# Matriz de correlação
corr_matrix = df[numeric_cols].corr()

# Visualização da matriz de correlação
plt.figure(figsize=(14, 10))
mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
sns.heatmap(corr_matrix, mask=mask, annot=True, fmt='.2f', cmap='coolwarm', vmin=-1, vmax=1, center=0)
plt.title('Matriz de Correlação das Variáveis Numéricas')
plt.tight_layout()
plt.show()

## Conclusões Preliminares

Após a análise exploratória, podemos tirar as seguintes conclusões preliminares:

1. **Distribuição dos dados**: [Adicione observações sobre as distribuições das principais variáveis]

2. **Valores ausentes**: [Adicione observações sobre padrões de valores ausentes]

3. **Variável alvo**: [Adicione observações sobre o balanceamento/desbalanceamento da variável alvo]

4. **Relações importantes**: [Adicione observações sobre relações significativas descobertas]

5. **Possíveis features importantes**: [Liste as variáveis que parecem ter maior poder preditivo]

6. **Próximos passos**: [Liste as etapas seguintes no processo de modelagem]
   - Tratamento de valores ausentes
   - Tratamento de outliers
   - Engenharia de features
   - Seleção de features
   - Modelagem e validação